このプログラムではではoriginの制御を行います

In [5]:
import originpro as op
import pandas as pd
import numpy as np
from math import *
#おまじない
# Ensures that the Origin instance gets shut down properly.
import sys
def origin_shutdown_exception_hook(exctype, value, traceback):
    op.exit()
    sys.__excepthook__(exctype, value, traceback)
if op and op.oext:
    sys.excepthook = origin_shutdown_exception_hook


# Set Origin instance visibility.
if op.oext:
    op.set_show(True)


In [6]:
xmin = 19
xmax = 180

In [7]:

# YOUR CODE HERE
#Book1を選択した状態で実行
#open origin file
op.open(file='C:/Users/mikku/OneDrive/デスクトップ/研究大学院/データ/MyFitting/data.opju')
BrillouinData = op.find_sheet('w','[Book1]BrillouinData') #get sheet
BrillouinData.name = 'BrillouinData'
# fit by Gaussamp
GaussFit = op.NLFit('Gaussamp')
GaussFit.set_data(BrillouinData,0,1)
GaussFit.fit()
#GaussFit.param_box()                 #収束の確認
GaussFit.report()                     #レポートの表示
GaussData = op.find_sheet('w','[Book1]FitNLCurve1') #GaussFitのsheetを取得
GaussData.name = 'GaussData'          #sheetの名前を変更
param = GaussFit.result()             #各パラメーターの表示
FWHM = param['FWHM']                  #FWHMの値を取得
print('FWHM = ',FWHM)

op.new_graph('GaussGraph')            #Gaussamp用のグラフを作成．[0],[1]がlayer
GaussGraph = op.find_graph('GaussGraph')
GaussGraph[0].add_plot(obj=BrillouinData,colx=0,coly=1)#add_plotはlayerに対してのobject
GaussGraph[0].add_plot(obj=GaussData,colx=0,coly=1).color = 'red'
GaussGraph[0].rescale()               #実際に描写する.必ずset_limの前に置く
GaussGraph[0].set_xlim(-5,5)          #(min,max,メモリの刻み)を返す

FWHM =  2.09350951895391


(-5.0, 5.0, 100.0)

In [8]:
#voigt関数でfitし，lorentz関数を抽出
Book2 = op.new_book('w','Book2')                       #新しいbookを作成
Brillouin_lim = op.find_sheet('w','[Book2]Sheet1')     
Brillouin_lim.name = 'Brillouin_lim'

df = BrillouinData.to_df()                             #dfに変換
df = df[(df['A']>xmin) & (df['A']<xmax)]                  #範囲を選択.&じゃないとダメみたい
Brillouin_lim.from_df(df)                              #dfをsheetに変換

op.new_graph('VoigtGraph')                             #Voigt用のグラフを作成
VoigtGraph = op.find_graph('VoigtGraph')
VoigtGraph[0].add_plot(obj=Brillouin_lim,colx=0,coly=1)
VoigtGraph[0].rescale()

AttributeError: 'NoneType' object has no attribute 'name'

######複数ピークフィットはできないので手動でvoigt関数でフィットしよう#########

In [ ]:
A1  =  6856.72361  #float(input("Enter the value for A1: "))
wL1 =  13.80626    #float(input("Enter the value for wL1: "))
xc1 =  70.07065    #float(input("Enter the value for xc1: "))
A2  =  4538.93155  #float(input("Enter the value for A2: "))
wL2 =  16.28969    #float(input("Enter the value for wL2: "))
xc2 =  126.59839   #float(input("Enter the value for xc2: "))

def Lorentz(x,A,wL,xc):
    return (2*A/pi)*(wL/(4*(x - xc)**2 + wL**2))     #ローレンツ関数

x = np.arange(xmin,xmax)
L1 = [Lorentz(x,A1,wL1,xc1) for x in x]
L2 = [Lorentz(x,A2,wL2,xc2) for x in x]
Lsum = [L1[i]+L2[i] for i in range(len(L1))]
data = {'x': x, 'L1': L1, 'L2': L2, 'Lsum': Lsum}
L = pd.DataFrame(data)
L.to_csv('data/86K_Lorentz.csv',index=True,sep=' ')

Book3 = op.new_book('w','Book3')                       #ローレンツ関数のプロットのための新しいbookを作成
LorentzData = op.find_sheet('w','[Book3]Sheet1')     
LorentzData.name = 'LorentzData'
LorentzData.from_df(L)

op.new_graph('LorentzGraph')                             #フォークト関数から抽出したLorentz関数のグラフを作成
LorentzGraph = op.find_graph('LorentzGraph')
LorentzGraph[0].add_plot(obj=LorentzData,colx=0,coly='Lsum')
LorentzGraph[0].rescale()

OSError: Cannot save file into a non-existent directory: 'data'

In [ ]:
# Exit running instance of Origin.
# 終了させるときに実行する．
if op.oext:
    op.exit()